
# 🧠 Deep Dive into `session.state` in Google ADK

This notebook is a **comprehensive and detailed breakdown** of the `session.state` object used in Google ADK (Agent Development Kit). The content here is derived from official documentation but is **paraphrased, visualized, and expanded** for clarity.

We'll cover:

1. What is `session.state`?
2. Key Characteristics of State
3. Organizing State with Prefixes (scope and persistence)
4. Accessing Session State in Agent Instructions
5. How State is Updated (recommended methods)
6. ⚠️ Warning: What NOT to do
7. Best Practices for State Design



## 1️⃣ What is `session.state`?

Think of `session.state` as the agent's **personal scratchpad** — a temporary memory bank for the conversation.

Unlike `session.events` (which stores the full interaction history), `session.state` stores **key-value pairs** representing useful information *during* a conversation:

| Use Case | Example |
|----------|---------|
| Personalization | `'user_theme': 'dark'` |
| Task Tracking | `'booking_step': 'confirm_payment'` |
| Information Accumulation | `'cart_items': ['book', 'pen']` |
| Flag Handling | `'user_is_authenticated': True` |

It's where the agent **reads and writes short-term facts** that influence ongoing behavior.



## 2️⃣ Key Characteristics of State

### ✅ Structure
- `session.state` is a dictionary (`dict` or `Map`).
- **Keys**: Must be strings.
- **Values**: Must be serializable (strings, numbers, booleans, simple lists/dicts).

⚠️ **Avoid** complex Python objects like functions, classes, or file handles.

### 🔁 Mutability
- The state is **expected to change** as the session evolves.

### 💾 Persistence
Depends on your SessionService:

| Service | Persistent? |
|---------|-------------|
| `InMemorySessionService` | ❌ Lost on restart |
| `DatabaseSessionService`, `VertexAiSessionService` | ✅ State survives restarts |



## 3️⃣ Organizing State with Prefixes: Scope Matters

To organize `session.state`, use **prefixes** to control **scope and persistence**:

| Prefix | Scope | Persists | Use Case |
|--------|-------|----------|----------|
| *(none)* | Current session | Depends on service | e.g. `'current_intent'` |
| `user:` | Specific user across all sessions | ✅ | e.g. `'user:language'` |
| `app:` | Shared across app/users | ✅ | e.g. `'app:config'` |
| `temp:` | Current *invocation* only | ❌ | e.g. `'temp:api_result'` |

**Note:** When sub-agents are called, the same `InvocationContext` (and `temp:` state) is shared.


```mermaid
flowchart TD
    A[App-level State app:] -->|Shared across users| AppService
    B[User-level State user:] -->|Shared across sessions| UserSession
    C[Session State] -->|Temporary session| ChatSession
    D[Temp State temp:] -->|One Invocation| LLM_Call

    AppService -->|Overrides| UserSession
    UserSession -->|Overrides| ChatSession
    ChatSession -->|Overrides| LLM_Call
    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style C fill:#e8f5e8
    style D fill:#fff3e0
    style AppService fill:#e3f2fd
    style UserSession fill:#f8e8f8
    style ChatSession fill:#e8f8e8
    style LLM_Call fill:#fff8e8
```


## 4️⃣ Accessing State in Agent Instructions

When defining an `LlmAgent`, you can inject values from `session.state` into the prompt using **curly-brace templating**:

```python
instruction="Tell a story about a {animal} who loves {hobby}."
```

If `session.state = {'animal': 'cat', 'hobby': 'coding'}`, this will become:

```
Tell a story about a cat who loves coding.
```

### 🛠 Fallback
Use `{key?}` if the key may be optional.

### ❗ Escaping `{}`
Use `InstructionProvider` to escape curly braces like `{{...}}` when needed.



## 5️⃣ How State is Updated: Recommended Methods

There are 3 recommended ways to update state:

### 1. `output_key` (Simple)
Automatically stores agent response into state.

```python
LlmAgent(..., output_key="last_response")
```

### 2. `EventActions.state_delta` (Manual)
For updating **multiple keys**, or setting scopes:

```python
EventActions(state_delta={
    "task_status": "done",
    "user:count": 2,
    "temp:progress": 0.75
})
```

### 3. From `CallbackContext` or `ToolContext`
Inside a tool or callback:

```python
context.state["user:count"] += 1
context.state["temp:status"] = "ok"
```

✅ This ensures changes are tracked and stored safely.



## ⚠️ A Warning About Direct State Modification

🚫 DO NOT do this:

```python
session = await session_service.get_session(...)
session.state["user:count"] = 42  # ❌ BAD PRACTICE
```

### Why?
- ❌ Not tracked in event history
- ❌ May NOT persist in DB-backed sessions
- ❌ Not thread-safe

✅ Instead, **always use**:
- `context.state[...]` in tools or callbacks
- `EventActions.state_delta` if manual
- `output_key` for simple responses



## ✅ Best Practices for State Design

- 🎯 **Minimal**: Store only what you need
- 🔐 **Serializable**: Use basic types (no custom classes)
- 🧩 **Prefixed**: `user:`, `app:`, `temp:` clearly define scope
- 📦 **Shallow**: Avoid deep nested structures
- ✅ **Use `append_event` or context.state for updates**
- 💡 **Design state like a contract** between your agents

---

This is your full guide to managing state in ADK like a pro! 💥
